In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import csv
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
driver = webdriver.Chrome(executable_path="D:\Download Default\chromedriver.exe")
url = "https://www.imdb.com/?ref_=nv_home"
driver.get(url)

C:\Users\Pc\AppData\Local\Temp/ipykernel_17964/3061250995.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="D:\Download Default\chromedriver.exe")


In [3]:
cols = ['movie id', 'movie title', 'release date', 'video release date',
              'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation',
              'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
              'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
              'Thriller', 'War', 'Western']
movies = pd.read_csv('./data/ml-100k/u.item', sep='|', names=cols, encoding='latin-1')

movie_titles = movies['movie title']

In [4]:
def format_title(title):
    start = title.find("(")
    if start == -1:
        return title
    end = title.find(")")
    if end == -1:
        return title
    
    check = title[start + 1:end]
    if check.isnumeric():
        return title[:end + 1]
    return title[:start] + title[end + 1:]




In [5]:
def getMovieDetaiPagelByTitle(title):
    search_field = driver.find_element_by_id('suggestion-search')
    if (title.find(")") != -1):
        format = format_title(title)
    search_field.send_keys(format)
    search_btn = driver.find_element_by_id('suggestion-search-button')
    search_btn.click()

    



In [6]:
from selenium.common.exceptions import NoSuchElementException
def getElementsBySelector(element, selector):
    try:
        element.find_elements_by_css_selector(selector)
    except NoSuchElementException:
        return ""
    return element.find_elements_by_css_selector(selector)

def getElementByXpath(element, Xpath):
    try:
        element.find_element_by_xpath(Xpath)
    except NoSuchElementException:
        return ""
    return element.find_element_by_xpath(Xpath)

In [7]:
def getExtraData(title):
    movie_url = getElementsBySelector(driver, ".result_text a")[0].get_attribute("href")
    driver.get(movie_url)  

    basic_details = getElementByXpath(driver, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div[1]/div[2]/ul')
    if basic_details == "":
        duration = ""
    else:
        li_info = getElementsBySelector(basic_details, 'li')
        if li_info == "":
            duration = ""
        else: 
            
            if len(li_info) < 3:
                duration = ""
            else:
                duration = li_info[2].text

    storyline_box = getElementsBySelector(driver, '[data-testid="Storyline"]')
    if storyline_box == "":
        storyline_content = ""
    else :
        print(storyline_box[0])
        storyline_content_box = getElementsBySelector(storyline_box[0], 'div.ipc-html-content')
        if storyline_content_box == "":
            storyline_content = ""
        else :
            storyline_content = getElementsBySelector(storyline_content_box[0], 'div')
            if storyline_content != "":
                storyline_content = storyline_content[0].text
    
    countries_box = getElementsBySelector(driver, '[data-testid="title-details-origin"]')
    if countries_box == "":
        contries = ""
    else:

        countries_url = getElementsBySelector(countries_box[0], 'a')
        if countries_url == "":
            countries = ""
        else:    
            countries = [country.text for country in countries_url]

    companies_box = getElementsBySelector(driver, '[data-testid="title-details-origin"]')
    if companies_box == "":
        contries = ""
    else:

        companies_url = getElementsBySelector(companies_box[0], 'a')
        if companies_url == "":
            companies = ""
        else:    
            companies = [company.text for company in companies_url]
    

    languages_box = getElementsBySelector(driver, '[data-testid="title-details-origin"]')
    if languages_box == "":
        contries = ""
    else:

        languages_url = getElementsBySelector(languages_box[0], 'a')
        if languages_url == "":
            languages = ""
        else:    
            languages = [language.text for language in languages_url]

    return {
        "title": title,
        "duration": duration,
        "storyline": storyline_content,
        "countries": countries,
        "companies": companies,
        "languages": languages
    }

    


In [8]:
with open('./data/extra-data/item.csv', 'a') as file_output:
    headers = ['movie title', 'duration', 'storyline', 'countries', 'production companies', 'languages']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n', fieldnames = headers)
    
    for title in movie_titles[:5]:
        getMovieDetaiPagelByTitle(title)
        extra_data = getExtraData(title)
        writer.writerow({
            headers[0]: extra_data["title"],
            headers[1]: extra_data["duration"],
            headers[2]: extra_data["storyline"],
            headers[3]: extra_data["countries"],
            headers[4]: extra_data["companies"],
            headers[5]: extra_data["languages"],
        })
        sleep(1)
        
        
        

C:\Users\Pc\AppData\Local\Temp/ipykernel_17964/2851152309.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_field = driver.find_element_by_id('suggestion-search')
C:\Users\Pc\AppData\Local\Temp/ipykernel_17964/2851152309.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_btn = driver.find_element_by_id('suggestion-search-button')
C:\Users\Pc\AppData\Local\Temp/ipykernel_17964/95701961.py:4: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  element.find_elements_by_css_selector(selector)
C:\Users\Pc\AppData\Local\Temp/ipykernel_17964/95701961.py:7: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  return element.find_elements_by_css_selector(selector)
C:\Users\Pc\AppData\Local\Temp/ipykernel_17964/95701961.py:11: DeprecationWarning: find_element_by_* commands are de

<selenium.webdriver.remote.webelement.WebElement (session="1d1749b7fa7d93f18db049ff16354b65", element="3c0dccb9-19a8-44f7-9eb5-94f9397ed8bf")>
<selenium.webdriver.remote.webelement.WebElement (session="1d1749b7fa7d93f18db049ff16354b65", element="66425f9c-85c6-4f02-84fc-c34d91f68256")>
<selenium.webdriver.remote.webelement.WebElement (session="1d1749b7fa7d93f18db049ff16354b65", element="c9c2d2a7-b08e-41d8-b57e-29a3c180e893")>
<selenium.webdriver.remote.webelement.WebElement (session="1d1749b7fa7d93f18db049ff16354b65", element="671b45dc-90ca-4914-8eae-dfe762702f19")>
<selenium.webdriver.remote.webelement.WebElement (session="1d1749b7fa7d93f18db049ff16354b65", element="16ac5fb6-cccf-4d89-b7e5-1ec74becba8b")>


In [9]:
s = "French Twist (Gazon maudit)"
s.split(" ")


['French', 'Twist', '(Gazon', 'maudit)']